                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-emz6k5rlwhzyisbhwa  xqfwvwdr-v0zs3_cbbe5xw   
1  BiTunyQ73aT9WBnpR9DZGw  oyogae7okpv6sygzt5g77q  7atyjtigm3jult4um3iypq   
2  saUsX_uimxRlCVr67Z4Jig  8g_imtfsiwikvnbp2etr0a  yjuwppi6hxg530lwp-fb2a   
3  AqPFMleE6RsU23_auESxiA  _7bhui9uuf5__hhc_q8guq  kxx2soes4o-d3zqbkimrfa   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbae6ddog4jkny91nclq  e4vwtrqf-wpjfwesgvdgxq   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      5       1      0     1   
2      3       0      0     0   
3      5       1      0     1   
4      4       1      0     1   

                                                text                date  \
0  If you decide to eat here, just be aware it is... 2018-07-07 22:09:11   
1  I've taken a lot of spin classes over the year... 2012-01-03 15:28:18   
2  Family diner. Had the buffet. Eclectic assortm... 2014-02-05 20:30:30   
3  Wow!  Yummy, different,  delici

In [ ]:
import pandas as pd

# Definir el tamaño de los chunks
chunk_size = 10000  # Ajusta según la capacidad de tu sistema

# Ruta del archivo JSON
file_path = r"C:\Users\yanin\OneDrive\Desktop\etl\data_lake\raw\yelp\review.json"

# Inicializar una lista para almacenar los DataFrames procesados
df_list = []

# Leer el archivo en chunks y procesar cada uno
for chunk in pd.read_json(file_path, lines=True, chunksize=chunk_size):
    # Convertir tipos de datos
    chunk = chunk.astype({
        'review_id': 'string',
        'user_id': 'string',
        'business_id': 'string',
        'stars': 'int',
        'date': 'datetime64[ns]',
        'useful': 'int',
        'funny': 'int',
        'cool': 'int'
    })

    # Crear columnas adicionales a partir de la fecha
    chunk['year'] = chunk['date'].dt.year
    chunk['month'] = chunk['date'].dt.month
    chunk['day_of_week'] = chunk['date'].dt.dayofweek
    chunk['day'] = chunk['date'].dt.day
    chunk['hour'] = chunk['date'].dt.hour

    # Filtrar reseñas con estrellas fuera de rango
    chunk = chunk[chunk['stars'].between(1, 5)]

    # Normalizar las columnas de ID
    chunk['user_id'] = chunk['user_id'].str.strip().str.lower()
    chunk['business_id'] = chunk['business_id'].str.strip().str.lower()

    # Agregar el chunk procesado a la lista
    df_list.append(chunk)

# Concatenar todos los chunks en un único DataFrame
df_reviews = pd.concat(df_list, ignore_index=True)


# Mostrar las primeras filas del DataFrame
print(df_reviews.head())

# Mostrar información general del DataFrame
print(df_reviews.info())
# Contar valores nulos por columna
null_counts = df_reviews.isnull().sum()
print("Valores nulos por columna:")
print(null_counts)

# Contar filas duplicadas
duplicate_counts = df_reviews.duplicated().sum()
print(f"Cantidad de filas duplicadas: {duplicate_counts}")
# Obtener dimensiones del DataFrame
shape = df_reviews.shape
print(f"Dimensiones del DataFrame: {shape[0]} filas y {shape[1]} columnas")
 #Guardar el DataFrame en formato Parquet
output_path = r"C:\Users\yanin\OneDrive\Desktop\proyecto final\archivos\yelp\reviews_processed.parquet"
df_reviews.to_parquet(output_path, engine='pyarrow', compression='snappy')

print(f"Archivo guardado como Parquet en: {output_path}")

Dimensiones del DataFrame: 6990280 filas y 14 columnas
